In [1]:
import numpy as np
import cv2
import pytesseract
from PIL import Image
import matplotlib.pyplot as plt
import os
from glob import glob
import time

In [2]:
def single_image_transform(input_image_path_name,languages):
        
        image = cv2.imread(input_image_path_name)
        
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        gray = cv2.bitwise_not(gray)
 
        thresh = cv2.threshold(gray, 0, 255,
                 cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
        coords = np.column_stack(np.where(thresh > 0))
        angle = cv2.minAreaRect(coords)[-1]
        
        
        if angle < -45:
            angle = -(90 + angle)
        else:
           angle = -angle
        
        
        (h, w) = image.shape[:2]
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        rotated = cv2.warpAffine(image, M, (w, h),
        flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
        
        ret,gray = cv2.threshold(rotated,150,255,cv2.THRESH_BINARY_INV)
        
        kernel = np.ones((1,1),np.float64)
        gray = cv2.erode(gray,kernel,iterations = 1)
        gray = cv2.morphologyEx(gray, cv2.MORPH_OPEN, kernel)
        gray = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, kernel)
        
        
        filename = input_image_path_name.replace('.TIF','_processed.TIF').format(os.getpid())
        cv2.imwrite(filename, gray)
        
        output_filename = input_image_path_name.replace('.TIF','.txt').format(os.getpid())
        
        text = pytesseract.image_to_string(Image.open(filename),lang=languages)
        
        text_file = open(output_filename, "w+",encoding='utf-8')
        
        text_file.write("%s" % text)
        text_file.close()
        
        

In [3]:
def multiple_image_transform(input_images_folder_path,languages):
    start_time = time.time()
    img_path = input_images_folder_path+"\*.TIF"
    img_names = glob(img_path)
    for filename in img_names:
        single_image_transform(filename,languages)
    print("Time taken to process "+ str(len(img_names))+" files: "+str(((time.time() - start_time)/60))+" minutes")   
    

In [6]:
def single_image_transform(input_image_path_name,languages):
        
        image = cv2.imread(input_image_path_name)
        
        
        output_filename = input_image_path_name.replace('.TIF','.txt').format(os.getpid())
        
        text = pytesseract.image_to_string(Image.open(input_image_path_name),lang=languages)
        
        text_file = open(output_filename, "w+",encoding='utf-8')
        
        text_file.write("%s" % text)
        text_file.close()
        

### Single image OCR:
#### Give input file name and languages
#### Processed image and output text file will be created in same folder

In [8]:
single_image_transform("my_results\input_images\img_five.TIF","asm+eng")

### Multiple images OCR:
#### Give input files folder path and languages(All files in a folder must have same language)
#### In below example,All files in a folder are asm+eng 

In [4]:
multiple_image_transform("my_results\input_images","asm+eng")

Time taken to process 2 files: 2.7062763929367066 minutes


In [3]:
from wand.image import Image

ImportError: MagickWand shared library not found.
You probably had not installed ImageMagick library.
Try to install:
  http://docs.wand-py.org/en/latest/guide/install.html#install-imagemagick-on-windows